In [1]:
import requests
import json 
import time 
import csv
import os
import re


In [2]:
def csv_writer(filename,to_write):
    headers = to_write.keys()
    file_exists = os.path.isfile(filename)

    with open(filename,'a') as f:
        writer = csv.DictWriter(f, delimiter=',',fieldnames=headers)
        
        
        if not file_exists:
            writer.writeheader()

        writer.writerow(to_write)
def execute_req(url, headers=None,params=None): 
    response = requests.request("GET", url, headers=headers, params=params)
    
    return json.loads(response.text)

def parse_tweets(jdata): 
    for tweet in jdata['data']: 
        yield tweet

        
def user_merger(tweets,users): 
    out = []
    for tweet in tweets:
        for user in users: 
            if tweet['author_id'] == user['id']:
                tweet['user_scr_name'] = user['username']
                tweet['username'] = user['name']
                out.append(tweet)
    
    return out




def id_search(id_string, dic_list):
    for e in dic_list :
        if e['id'] == id_string:
            return e


def rt_merger(tweets,extended): 
    out =[]
    for tweet in tweets:
        tweet['retweeted_status']=''
        if 'referenced_tweets' in tweet and tweet['referenced_tweets'][0]['type']=='retweeted': 
            id_s = tweet['referenced_tweets'][0]['id']
   
            out_r = id_search(id_s,extended)
            tweet['retweeted_status'] = out_r['text']
            tweet['retweet_count'] = out_r['public_metrics']['retweet_count'] 
            tweet['favorite_count'] = out_r['public_metrics']['like_count']
            tweet['comm_count'] = out_r['public_metrics']['quote_count'] 

        out.append(tweet)
            
       
    return out


def rt_user_merger(tweets, auts):
    for tweet in tweets:
        if tweet['text'].startswith('RT'):
            try: 
             rt_user = re.findall('(?<=RT\s)@\S+', tweet['text'])[0]
             rt_user = rt_user.strip('@')
             rt_user = rt_user.strip(':')

             for a in auts:
                if a['username'] ==rt_user:
       
                    tweet['rt_user'] = rt_user
                    tweet['rt_followers'] = a['public_metrics']['followers_count']
                    break
            except: 
             pass

    return tweets
        




def assemble_tweet(tweet_d):
    out={}
    fields_e = fields_l + ['user_scr_name','username','retweeted_status','rt_followers', 'retweet_count','comm_count','favorite_count']
    
    for f in fields_e:
        if f in tweet_d: 
            out[f] = tweet_d[f]
        else: 
            out[f] = ''
            

    
    try:
        out['exp_url'] = tweet_d['entities']['urls'][0]['expanded_url']
    except:
        out['exp_url']='n/a'
    
    
   
    
    return out

def process_tweets(j_response): 

        

        tweets=j_response['data']
    

        users = j_response['includes']['users']
        extended = j_response['includes']['tweets']
        tweets = user_merger(tweets,users)

        tweets = rt_merger(tweets,extended)
        tweets = rt_user_merger(tweets,users)

        tweets = [assemble_tweet(tweet) for tweet in tweets]
        
        print(tweets[1])
        return tweets

def expand_text(tweet): 
    if tweet['retweeted_status']: 
        tweet['rt_aut'] = re.findall(r'RT @(\w+):',tweet['text'])[0]
        
        
        tweet['text'] = 'RT @' + tweet['rt_aut'] +':'+tweet['retweeted_status'].replace('\n','')
        return tweet
    else: 
        tweet['rt_aut'] = ''
        tweet['text'] = tweet['text'].replace('\n','')
        return tweet
    
def save_tweets(filename, tweets): 
    tweets = [expand_text(tweet) for tweet in tweets]

    for tweet in tweets: 
        csv_writer(filename, tweet)
        

        
        
def entity_tweet_network(tweet): 
        if tweet['context_annotations']: 
            edges=[]
            annotations = tweet['context_annotations']
            
            unique_ids = list(set([annotation['entity']['id']  for annotation in annotations]))
            
            if len(unique_ids)>1:
                processed_ids =[]
                for unique_id in unique_ids:
                    for annotation in annotations: 
                        if annotation['entity']['id'] == unique_id and  unique_id not in processed_ids: 
                            edge={}
                            edge['tweet_id'] = tweet['id']
                            edge['annot_id'] = annotation['entity']['id'] 
                            edge['annot_name'] = annotation['entity']['name']

                            edges.append(edge)
                            processed_ids.append(unique_id)
                            
                
            else: 
                edge={}
                edge['tweet_id'] = tweet['id']
                edge['annot_id'] = annotations[0]['entity']['id'] 
                edge['annot_name'] = annotations[0]['entity']['name']

                edges.append(edge)
        
        
        
            return edges
        
        else:
            return []

    

In [3]:
fields_l = ['author_id','created_at','text','geo','lang','public_metrics','id','entities','context_annotations']
fields = ','.join(fields_l)


In [4]:
query =   'covid lang:it' 
filename=  'test.csv'

bearer_token = ''

query_params = {'query': query,
                'tweet.fields':fields , 
                'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                'expansions':'author_id,referenced_tweets.id,referenced_tweets.id.author_id',
                'max_results':100, 
                'start_time':'2021-01-01T00:00:00Z',
                'end_time':'2021-01-10T00:00:00Z'}
n_results=4000


In [5]:
headers = {"Authorization": "Bearer {}".format(bearer_token)}
search_url = "https://api.twitter.com/2/tweets/search/all"






#data_out=[]


pages = int(n_results/query_params['max_results'])

next_token=False

for i, page in enumerate(range(pages)):
    print(page)
    
    time.sleep(4)
    
    
    
    
    if not next_token: 
        
        j_response = execute_req(search_url, headers=headers, params=query_params)
       
        tweets=process_tweets(j_response)
        

        try: 
            next_token = j_response['meta']['next_token']
        except: 
            break
    else:
        updated_params = query_params
        updated_params['next_token'] = next_token
        j_response = execute_req(search_url, headers=headers, params=updated_params)
        
        tweets=process_tweets(j_response)
 



        try: 
            next_token = j_response['meta']['next_token']
        except: 
            break
        
        



    print(len(tweets))
    
    save_tweets(filename,tweets)
    


0
{'author_id': '2002721', 'created_at': '2021-01-09T23:59:35.000Z', 'text': 'RT @janavel7: Parlare chiaro.\nIl Papa: “Farò il vaccino anti-Covid. C’è un negazionismo suicida”.\nAmen', 'geo': '', 'lang': 'it', 'public_metrics': {'retweet_count': 130, 'reply_count': 0, 'like_count': 0, 'quote_count': 0, 'impression_count': 0}, 'id': '1348056858578325505', 'entities': {'mentions': [{'start': 3, 'end': 12, 'username': 'janavel7', 'id': '2204706469'}]}, 'context_annotations': [{'domain': {'id': '123', 'name': 'Ongoing News Story', 'description': "Ongoing News Stories like 'Brexit'"}, 'entity': {'id': '1220701888179359745', 'name': 'COVID-19'}}, {'domain': {'id': '131', 'name': 'Unified Twitter Taxonomy', 'description': 'A taxonomy of user interests. '}, 'entity': {'id': '1220701888179359745', 'name': 'COVID-19'}}], 'user_scr_name': 'dasar', 'username': 'dasar', 'retweeted_status': 'Parlare chiaro.\nIl Papa: “Farò il vaccino anti-Covid. C’è un negazionismo suicida”.\nAmen', 'rt_followers': 

KeyboardInterrupt: 

In [8]:
tweets = [expand_text(tweet) for tweet in tweets]

for tweet in tweets: 
        csv_writer(filename, tweet)

In [25]:
#  extract annotations 

filename_annotations= 'test_entities.csv'
all_edges=[]
for tweet in tweets: 
    edges = entity_tweet_network(tweet)
    all_edges.extend(edges)

        
        
        
        
        
        
        
        
        

        
        
        

        
        
        
        

        

In [27]:
tweets[1]

{'author_id': '317671787',
 'created_at': '2021-01-09T18:46:57.000Z',
 'text': 'In Italia 19.978 nuovi casi, 483 decessi Covid, Veneto regione col numero più alto https://t.co/e2sBMSwRNe',
 'geo': '',
 'lang': 'it',
 'public_metrics': {'retweet_count': 0,
  'reply_count': 0,
  'like_count': 0,
  'quote_count': 0},
 'id': '1347978182729867267',
 'entities': {'urls': [{'start': 83,
    'end': 106,
    'url': 'https://t.co/e2sBMSwRNe',
    'expanded_url': 'https://www.informazione.it/a/34CA84A0-CD71-4268-B887-27D164EE17FB/In-Italia-19-978-nuovi-casi-483-decessi-Covid-Veneto-regione-col-numero-piu-alto',
    'display_url': 'informazione.it/a/34CA84A0-CD7…'}]},
 'context_annotations': [{'domain': {'id': '123',
    'name': 'Ongoing News Story',
    'description': "Ongoing News Stories like 'Brexit'"},
   'entity': {'id': '1220701888179359745', 'name': 'COVID-19'}}],
 'user_scr_name': 'infoitinterno',
 'username': 'informazione interno',
 'retweeted_status': '',
 'rt_followers': '',
 'retweet

In [110]:
len([0])

1